## Inicialização

In [1]:
import numpy as np
import pandas as pd
from mlfluids import Fluid, set_refprop_path

set_refprop_path("/home/braun/Documents/Developer/REFPROP10")

## Cálculo das propriedades

In [2]:
MIXTURE = "Nitrogen&Oxygen"
PHASE = "supercritical"
PROPS = ["PHASE", "P*", "T*", "VIS*", "TCX*"]

In [3]:
fluid = Fluid(MIXTURE)

X = np.linspace(0, 1, 10)[1:]

Tr = np.geomspace(1.1, 10, 10)
Pr = np.geomspace(1.1, 20, 10)

Pv, Tv = np.meshgrid(Pr, Tr, indexing="ij")
PrTr = np.column_stack((Pv.ravel(), Tv.ravel()))


In [5]:
norm_fluid = Fluid(MIXTURE)

def calc_props_X(Xv: float) -> pd.DataFrame:
    # SETUP: FLUIDO DE REFERÊNCIA PARA NORMALIZAÇÃO
    norm_fluid.set_mole_fractions([Xv, 1 - Xv])
    norm_fluid.set_state_critical()
    Fluid.set_normalizing_fluid(norm_fluid)

    # SETUP: FLUIDO DE ANÁLISE
    fluid.set_mole_fractions([Xv, 1 - Xv])
    fluid.specify_phase(PHASE)
    fluid._D = norm_fluid.rhomolar_critical()

    # CÁLCULO DE PARÂMETROS
    props_data = fluid.calc_states_props(
        "P*", "T*", PrTr, PROPS, log_errors=False
    )

    return pd.DataFrame(props_data, columns=PROPS)

results = pd.concat([calc_props_X(Xv) for Xv in X], ignore_index=True)

display(results)


,PHASE,P*,T*,VIS*,TCX*
0,supercritical,1.1,1.1,0.630056,0.32776
1,supercritical,1.1,1.405739,0.692847,0.309177
2,supercritical,1.1,1.796456,0.818947,0.354482
3,supercritical,1.1,2.29577,0.973863,0.420896
4,supercritical,1.1,2.933867,1.15604,0.505385
...,...,...,...,...,...
855,supercritical,20.0,3.749318,1.978079,0.00004
856,supercritical,20.0,4.79142,2.053665,0.000041
857,supercritical,20.0,6.123168,2.222316,0.000044
858,supercritical,20.0,7.825067,2.468966,0.000049
